<a href="https://colab.research.google.com/github/Chachoummm/chess_engine/blob/main/Train_Evaluate_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#libraries
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [6]:
rm -rf ./logs/

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
# comme c'est trop lourd c'est pas sur github donc le modèle a été entrainé en local et puis on y a fait appel sur colab à partir de Github
data= pd.read_csv('/content/gdrive/MyDrive/filtered_2100_ranking (2).csv', index_col=None, header=0)

In [11]:
data = shuffle(data)
print(data.shape)
print(data.head())

(610240, 193)
          a1    b1    c1    d1    e1  ... to_e8 to_f8 to_g8 to_h8 good_move
601168  None  None  None  None  None  ...   0.0   0.0   0.0   0.0     False
563510  None  None  None  None     R  ...   0.0   0.0   0.0   0.0     False
62442   None  None  None     R  None  ...   0.0   0.0   0.0   0.0     False
574307     R  None     B     Q     K  ...   0.0   0.0   0.0   0.0     False
274284  None  None     K     R  None  ...   0.0   0.0   0.0   0.0     False

[5 rows x 193 columns]


In [12]:
#features transformations 
features = list(data.iloc[:, 0:192].columns)
X = data[features]
y = data['good_move']
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
  vocabulary = X[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in numerical_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

In [13]:
# Split the data into train set (0.8) and test set (0.2)
from sklearn.model_selection import train_test_split
train_set, test_set= train_test_split(data, test_size=0.2, random_state=0, shuffle=True)
X_train,y_train, X_test,y_test = train_test_split(X,y, test_size=0.2, random_state=0, shuffle=True)

# Verify the plits
#print(train_set.shape, test_set.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(488192, 192) (122048, 192) (488192,) (122048,)


In [ ]:
# TRAINING PART

In [15]:
# Split train_set into batches :  
  
def split_into_batches(df, batch_size=100000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  train_batches_X = []
  train_batches_y = []
  
  for i in range(0, len(intervals) - 1):
    train_batches_X.append(train_set.iloc[intervals[i]:intervals[i + 1], :][features])
    train_batches_y.append(train_set.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return train_batches_X, train_batches_y

train_batches_X, train_batches_y = split_into_batches(train_set)

In [ ]:
print(train_batches_X)
print(train_batches_y)

In [18]:
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='/content/gdrive/MyDrive/Saved_good_model',
    warm_start_from=None, loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE,
    sparse_combiner='sum'
  )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/gdrive/MyDrive/Saved_good_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
# Create the input function 
  
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function 

In [20]:
# Create a list with all the train input function : 
train_input_functions = []
for df_X, df_y in zip(train_batches_X, train_batches_y):
  train_input_functions.append(make_input_fn(df_X, df_y))

print(len(train_input_functions))

5


In [ ]:
# train the model on all the input functions
i = 1
for input_function in train_input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  linear_est.train(input_function)
  

<======================================== NEW BATCH ========================================>
Batch: 1
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /content/gdrive/MyDrive/Saved_good_model/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 23.7943
INFO:tensorflow:loss = 0.13955602, step = 100 (4.207 sec)
INFO:tensorflow:global_step/sec: 84.0184
INFO:tensorflow:loss = 0.032262765, step = 200 (1.188 sec)
INFO:tensorflow:global_step/sec: 79.9294
INFO:tensorflow:loss = 0.35488048, step = 300 (1.255 sec)
INFO:tensorflow:global_step/sec: 81.7876
INFO:tensorflow:loss = 0.031557284, step = 400 (1

In [ ]:
# EVALUATION PART

In [ ]:
# Split the test_set into batches :  
  
def split_into_batches(df, batch_size=50000):
  nb_rows = len(df.index)
  intervals = []
  
  for i in range(0, nb_rows + 1, batch_size):
    intervals.append(i)
  
  if(intervals[-1] != nb_rows):
    intervals.append(nb_rows)
  
  test_batches_X = []
  test_batches_y = []
  
  for i in range(0, len(intervals) - 1):
    test_batches_X.append(test_set.iloc[intervals[i]:intervals[i + 1], :][features])
    test_batches_y.append(test_set.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

  return test_batches_X, test_batches_y

test_batches_X, test_batches_y = split_into_batches(test_set)

In [ ]:
# Create a list with all the test input functions 
test_input_functions = []
for df_X, df_y in zip(test_batches_X, test_batches_y):
  test_input_functions.append(make_input_fn(df_X, df_y)

print(len(test_input_functions))

In [ ]:
# Evaluate the model on all the inpunt functions (Give score for the test i guess)
i = 1
for input_function in test_input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  model_eval =linear_est.evaluate(input_function)


In [ ]:
loss_score = model_eval["loss"]
print("Loss: {0:f}".format(loss_score))

In [ ]:
# Check the summary statistic to get an idea of how big the error is.
training_set['medv'].describe()

In [ ]:
# PREDICTION PART 

In [ ]:
# Create a list with all the pred input functions 
pred_input_functions = []
for df_X in test_batches_X :
  pred_input_functions.append(make_pred_input_fn(df_X))

print(len(pred_input_functions))

In [ ]:
# Create the prediction input function

def make_pred_input_fn(data_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function        

In [ ]:
# The model prediction on all the inpunt functions 
i = 1
for input_function in pred_input_functions:
  print('<======================================== NEW BATCH ========================================>')
  print('Batch: ' + str(i))
  i = i + 1
  y_predict = estimator.predict( pred_input_fn)

In [ ]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))

In [ ]:
print("Predictions: {}".format(str(predictions)))

In [ ]:
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = '/content/gdrive/MyDrive/Saved_train_model'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

In [ ]:
!tensorboard --logdir='\content\gdrive\MyDrive\Model_train_Ourdata' \
  --name "Projet DATAB" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  #--one_shot

2021-12-08 14:30:40.445803: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/OUMaqezDRxeeKIY6KGOGdg/

[2021-12-08T14:30:40] Started scanning logdir.


Interrupted.
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/pro

In [ ]:


import keras
from matplotlib import pyplot as plt
history = model.fit()

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()



plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()